In [1]:
import requests

The data for Boone County's COVID dashboard is coming from the [REST API](https://developers.arcgis.com/rest/) of their ArcGIS server instance:

- The identifer for that server is `GHhNHT1xiCkCAXvo`
- The ["feature service"](https://developers.arcgis.com/rest/services-reference/feature-service.htm) for that data is `COVID19_Data`
- The first and only ["layer"](https://developers.arcgis.com/rest/services-reference/layer-feature-service-.htm) of this service is a sheet (i.e., in the `0` position)

So here is the URL to query the first data sheet:

In [2]:
url = "https://services.arcgis.com/GHhNHT1xiCkCAXvo/arcgis/rest/services/COVID19_Data/FeatureServer/0/query"

To fetch the data, we need to make a `GET` request with the appropriate query string parameters.

This is roughly what we found in one of those requests using my web browser's development tools. Note that I modified the `'outfields'` to value so as to remove fields that don't contain any values for this particular query.

In [3]:
params = {
    "f": "json",
    "where": "Date_of_Test BETWEEN timestamp '2020-09-17 05:00:00' AND CURRENT_TIMESTAMP",
    "returnGeometry": False,
    "spatialRel": "esriSpatialRelIntersects",
    "outFields": [
        'Date_of_Test',
        'Daily_Cases',
        'Cumulative_Cases',
        'Percent_Increase',
        'Five_Day_Average',
        'Date_of_Actual_Death',
        'Deaths_Totals',
        'FID',
    ],
    "orderByFields": "Date_of_Test desc",
    "resultOffset": 0,
    "resultType": "standard" 
}

In [4]:
r = requests.get(url, params=params)

We can also see (and modify) this query via the ArcGIS REST services [web interface](https://services.arcgis.com/GHhNHT1xiCkCAXvo/arcgis/rest/services/COVID19_Data/FeatureServer/0/query?f=html&where=Date_of_Test+BETWEEN+timestamp+%272020-09-17+05%3A00%3A00%27+AND+CURRENT_TIMESTAMP&returnGeometry=False&spatialRel=esriSpatialRelIntersects&outFields=Date_of_Test&outFields=Daily_Cases&outFields=Cumulative_Cases&outFields=Percent_Increase&outFields=Five_Day_Average&outFields=Date_of_Actual_Death&outFields=Deaths_Totals&outFields=FID&orderByFields=Date_of_Test+desc&resultOffset=0&resultType=standard)

The data is returned in the json of the response. Because ArcGIS is built for geo-spatial data, each observation (aka, "record" or "row") is labeled a ["feature"](https://developers.arcgis.com/rest/services-reference/feature-feature-service-.htm).

Here are the first five.

In [5]:
r.json()['features'][:5]

[{'attributes': {'Date_of_Test': 1608127200000,
   'Daily_Cases': 124,
   'Cumulative_Cases': 12254,
   'Percent_Increase': 0.010119144769055,
   'Five_Day_Average': 108,
   'Date_of_Actual_Death': 1608127200000,
   'Deaths_Totals': 40,
   'FID': 55}},
 {'attributes': {'Date_of_Test': 1608040800000,
   'Daily_Cases': 80,
   'Cumulative_Cases': 12130,
   'Percent_Increase': 0.00659521846661171,
   'Five_Day_Average': 107,
   'Date_of_Actual_Death': 1608040800000,
   'Deaths_Totals': 40,
   'FID': 54}},
 {'attributes': {'Date_of_Test': 1607954400000,
   'Daily_Cases': 91,
   'Cumulative_Cases': 12050,
   'Percent_Increase': 0.00755186721991701,
   'Five_Day_Average': 111,
   'Date_of_Actual_Death': 1607954400000,
   'Deaths_Totals': 40,
   'FID': 53}},
 {'attributes': {'Date_of_Test': 1607868000000,
   'Daily_Cases': 129,
   'Cumulative_Cases': 11959,
   'Percent_Increase': 0.0107868550882181,
   'Five_Day_Average': 134,
   'Date_of_Actual_Death': 1607868000000,
   'Deaths_Totals': 40,
 

Above, we see a list that contains a dict with just one key (`"attributes"`) with a value that is another, inner dict.

Typically, each feature would have both a geometry representing the shape of that geography and attributes of the geography (e.g., the name and populations of each county in a layer). However, in this case, it's all just attributes. So you might as well remove that extra key.

In [6]:
data = [d['attributes'] for d in r.json()['features']]

In [7]:
data[:5]

[{'Date_of_Test': 1608127200000,
  'Daily_Cases': 124,
  'Cumulative_Cases': 12254,
  'Percent_Increase': 0.010119144769055,
  'Five_Day_Average': 108,
  'Date_of_Actual_Death': 1608127200000,
  'Deaths_Totals': 40,
  'FID': 55},
 {'Date_of_Test': 1608040800000,
  'Daily_Cases': 80,
  'Cumulative_Cases': 12130,
  'Percent_Increase': 0.00659521846661171,
  'Five_Day_Average': 107,
  'Date_of_Actual_Death': 1608040800000,
  'Deaths_Totals': 40,
  'FID': 54},
 {'Date_of_Test': 1607954400000,
  'Daily_Cases': 91,
  'Cumulative_Cases': 12050,
  'Percent_Increase': 0.00755186721991701,
  'Five_Day_Average': 111,
  'Date_of_Actual_Death': 1607954400000,
  'Deaths_Totals': 40,
  'FID': 53},
 {'Date_of_Test': 1607868000000,
  'Daily_Cases': 129,
  'Cumulative_Cases': 11959,
  'Percent_Increase': 0.0107868550882181,
  'Five_Day_Average': 134,
  'Date_of_Actual_Death': 1607868000000,
  'Deaths_Totals': 40,
  'FID': 52},
 {'Date_of_Test': 1607781600000,
  'Daily_Cases': 114,
  'Cumulative_Cases': 

In [8]:
timestamp = data[0]['Date_of_Test']

In [9]:
from datetime import datetime

In [10]:
timestamp_no_ms = int(str(timestamp)[:-3])

In [11]:
naive_dt = dt = datetime.fromtimestamp(timestamp_no_ms)

In [12]:
print(naive_dt)

2020-12-16 08:00:00


See [Aware versus Naive Objects](https://docs.python.org/3/library/datetime.html#aware-and-naive-objects) in Python's `datetime` module docs.

It's typically a good idea to include timezone info in your datetime values. There are several third-party Python packages to make this easier (so `pip install pytz` to make this work below).

You want to be careful on this front, though. Make sure you are clear with the source about the timezone context (including daylight savings vs not) in which the data is being recorded.

In [13]:
import pytz

In [14]:
central = pytz.timezone('US/Central')

In [15]:
dt = datetime.fromtimestamp(timestamp_no_ms, tz=central)

In [16]:
print(dt)

2020-12-16 08:00:00-06:00
